In [1]:
import sqlite3
from custom_agents.chat_planner_agent import chat_planner_agent
from transformers import AutoTokenizer
import os
from langchain_groq import ChatGroq

import google.generativeai as genai


def get_llm():

    api_key = os.environ['GROQ_API_KEY']
    llm = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama-3.1-70b-versatile")

    return llm

def get_long_ctx_llm():

    api_key = os.environ['GEMINI_API_KEY']
    llm = genai.configure(api_key=api_key)

    return llm

def get_chat_expert(_llm, _tokenizer, _planning_llm, _long_ctx_llm):
    return chat_planner_agent(_llm,_tokenizer, _planning_llm, _long_ctx_llm, log_level='DEBUG', log_file='./agentlogs/chat_planner.txt', logging_enabled=True  )

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

llm = get_llm()
long_ctx_llm = get_long_ctx_llm()
planning_llm = llm
chat_expert = get_chat_expert(llm,tokenizer, planning_llm, long_ctx_llm)


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [13]:
chat_history

[{'role': 'assistant',
  'content': 'Hello, ask me something about the music sales db'},
 {'role': 'user', 'content': 'artist that sold more units by year '},
 {'role': 'assistant',
  'content': ('Based on the data, here are the top artists by year who sold the most units:\n\n* 2013: Iron Maiden (36 units)\n* 2012: Iron Maiden (34 units)\n* 2010: Iron Maiden (35 units)\n* 2009: Iron Maiden (34 units)\n\nIt seems Iron Maiden was the top-selling artist in these years. Would you like to know more about a specific year or artist?',
                      Name  year  total_units
   0           Iron Maiden  2013           36
   1           Iron Maiden  2010           35
   2           Iron Maiden  2009           34
   3           Iron Maiden  2012           34
   4                    U2  2012           28
   ..                  ...   ...          ...
   548     Velvet Revolver  2013            1
   549  Vinícius De Moraes  2010            1
   550  Vinícius De Moraes  2012            1
   551

In [17]:

chat_history = [{"role":"assistant","content":"Hello, ask me something about the music sales db"}]
message = "Muestrame los tracks de Iron Maiden "
chat_history.append({"role":"user","content":message})
query = message

In [12]:

message = "mostrame solo el top 10"
chat_history.append({"role":"user","content":message})
query = message

In [18]:
expert_answer = chat_expert.ask_question({"messages": chat_history,"query":query})
print("#09")
chat_history.append({"role":"assistant","content":expert_answer})

for message in chat_history:
    print(message['role'], message['content'])
print("#10")

#200 init agent chat **
#debug begin init db  18:14:57
#debug begin analyze doc   18:14:57
#debug end analyze doc   18:14:58
#debug begin exe plan db expert  18:14:58
#debug init action    18:14:58
#debug end action    18:14:58
#07  df_1
#debug end plan    18:14:58
#debug end x plan after gen from db    18:14:58
#007 db expert ask question par state  {'question': 'Get tracks by artist Iron Maiden'}
#100 result from db  Get tracks by artist Iron Maiden * Answer from expert:
(results limited to first 15 rows. Here you have a limited list, please use with caution because it's incomplete, inform the user about that and don't use the following information to do answer follow up questions, instead re write your question)
* Columns of the report: Name
Data from the report:
Different World
These Colours Don't Run
Brighter Than a Thousand Suns
The Pilgrim
The Longest Day
Out of the Shadows
The Reincarnation of Benjamin Breeg
For the Greater Good of God
Lord of Light
The Legacy
Hallowed Be Thy N

In [9]:


# Conectar a la base de datos
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = conn.cursor()


In [4]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = conn.cursor()

# Get the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Generate the schema description
schema_description = ""
for table in tables:
    table_name = table[0]
    schema_description += f"{table_name}("
    
    # Get the columns
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    column_names = [column[1] for column in columns]
    schema_description += ", ".join(column_names) + ")\n"
    
    # Get the primary keys
    primary_keys = [column[1] for column in columns if column[5] == 1]
    if primary_keys:
        schema_description += f"  Primary Key: {', '.join(primary_keys)}\n"
    
    # Get the foreign keys
    cursor.execute(f"PRAGMA foreign_key_list({table_name});")
    foreign_keys = cursor.fetchall()
    if foreign_keys:
        schema_description += "  Foreign Keys:\n"
        for fk in foreign_keys:
            schema_description += f"    - {fk[3]} -> {fk[2]}({fk[4]})\n"
    schema_description += "\n"

# Close the connection
conn.close()

# Print the schema description
print(schema_description)


Album(AlbumId, Title, ArtistId)
  Primary Key: AlbumId
  Foreign Keys:
    - ArtistId -> Artist(ArtistId)

Artist(ArtistId, Name)
  Primary Key: ArtistId

Customer(CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId)
  Primary Key: CustomerId
  Foreign Keys:
    - SupportRepId -> Employee(EmployeeId)

Employee(EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email)
  Primary Key: EmployeeId
  Foreign Keys:
    - ReportsTo -> Employee(EmployeeId)

Genre(GenreId, Name)
  Primary Key: GenreId

Invoice(InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total)
  Primary Key: InvoiceId
  Foreign Keys:
    - CustomerId -> Customer(CustomerId)

InvoiceLine(InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity)
  Primary Key: InvoiceLineId
  Foreign Keys:
    - TrackId -> Track(TrackId)
    

In [6]:
# Consulta SQL con tablas en FROM y condiciones en WHERE
query = '''
    SELECT ar.Name AS ArtistName, SUM(il.UnitPrice * il.Quantity) AS TotalRevenue
    FROM InvoiceLine il, Track t, Album al, Artist ar
    WHERE il.TrackId = t.TrackId
    AND t.AlbumId = al.AlbumId
    AND al.ArtistId = ar.ArtistId
    GROUP BY ar.ArtistId
    ORDER BY TotalRevenue DESC;
'''

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
artists_revenue = cursor.fetchall()

# Mostrar los resultados
print("Artistas que más han facturado (de mayor a menor):")
for artist, revenue in artists_revenue:
    print(f"{artist}: ${revenue:.2f}")




Artistas que más han facturado (de mayor a menor):
Iron Maiden: $138.60
U2: $105.93
Metallica: $90.09
Led Zeppelin: $86.13
Lost: $81.59
The Office: $49.75
Os Paralamas Do Sucesso: $44.55
Deep Purple: $43.56
Faith No More: $41.58
Eric Clapton: $39.60
R.E.M.: $38.61
Creedence Clearwater Revival: $36.63
Queen: $36.63
Battlestar Galactica (Classic): $35.82
Guns N' Roses: $35.64
Titãs: $33.66
Green Day: $32.67
Pearl Jam: $31.68
Kiss: $30.69
Van Halen: $28.71
Various Artists: $28.71
Red Hot Chili Peppers: $26.73
Chico Buarque: $26.73
Heroes: $25.87
Lenny Kravitz: $25.74
Chico Science & Nação Zumbi: $24.75
Battlestar Galactica: $23.88
Tim Maia: $23.76
Smashing Pumpkins: $23.76
Ozzy Osbourne: $23.76
Foo Fighters: $21.78
Antônio Carlos Jobim: $21.78
Cássia Eller: $20.79
Caetano Veloso: $20.79
Spyro Gyra: $19.80
Gilberto Gil: $19.80
The Who: $18.81
Amy Winehouse: $17.82
The Rolling Stones: $17.82
Jamiroquai: $17.82
Djavan: $17.82
The Tea Party: $16.83
Legião Urbana: $16.83
Gene Krupa: $16.83
Mil

#01
#02 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
#02
#03
#04
#45
#05
#06
#07
#08


#77 ask question external par state  {'messages': [{'role': 'assistant', 'content': 'Hello, ask me something.'}, {'role': 'user', 'content': 'cuantos son los signos del zodiaco?'}, {'role': 'assistant', 'content': 'Una pregunta interesante, pero no relacionada con tu carta natal. Hay 12 signos del zodiaco, pero para poder ayudarte de manera más personalizada, necesito saber más sobre ti. ¿Cuál es tu fecha de nacimiento?'}], 'query': 'consejo de vida', 'birthchart': 'vacio'}
#63 last messagest checked <|start_header_id|>assistant<|end_header_id|>
Hello, ask me something.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
cuantos son los signos del zodiaco?
<|eot_id|>

#64 query sent  consejo de vida
#88 check result  La respuesta es 12. Hay 12 signos del zodiaco. 

*** PREGUNTA DEL USUARIO
 consejo de vida
*** FIN PREGUNTA DEL USUARIO

La tarea es determinar si la pregunta del usuario es relativa a astrología.

La respuesta es: PASS  ------  
#11
#12 <|start_header_id|>assistant<|end_h

In [7]:
# Cerrar la conexión
conn.close()

In [24]:
from langchain_core.output_parsers import JsonOutputParser

# Tu string JSON
json_string = """[
{
    "step": 1,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "artist",
      "df_b": "album",
      "on": "ArtistId",
      "how": "inner"
    },
    "save_as": "df_1"
  },
  {
    "step": 2,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "df_1",
      "df_b": "track",
      "on": "AlbumId",
      "how": "inner"
    },
    "save_as": "df_2"
  }  ,
  {
    "step": 3,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "df_2",
      "df_b": "invoice_line",
      "on": "TrackId",
      "how": "inner"
    },
    "save_as": "df_3"
  },
  {
    "step": 4,
    "action": "OP_GROUP_BY",
    "parameters": {
      "df_a": "df_3",
      "group_by_columns": ["artist.Name"],
      "agg_functions": ["SUM(invoice_line.Quantity * track.UnitPrice) AS total_revenue"]
    },
    "save_as": "df_4"
  }
  
  ]
  """
a= """,
,
  {
    "step": 5,
    "action": "OP_ORDER",
    "parameters": {
      "df_a": "df_4",
      "columns": ["total_revenue"],
      "ascending": [False]
    },
    "save_as": "df_5"
  },
  {
    "step": 6,
    "action": "OP_LIMIT",
    "parameters": {
      "df_a": "df_5",
      "limit": 10
    },
    "save_as": "df_6"
  }
]
"""
# Crea una instancia del JsonOutputParser
parser = JsonOutputParser()

# Parsear el string JSON
try:
    resultado = parser.parse(json_string)
    print(resultado)  # Esto debería imprimir un diccionario de Python
except Exception as e:
    print(f"Error al parsear JSON: {e}")


Error al parsear JSON: Invalid json output: [
{
    "step": 1,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "artist",
      "df_b": "album",
      "on": "ArtistId",
      "how": "inner"
    },
    "save_as": "df_1"
  },
  {
    "step": 2,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "df_1",
      "df_b": "track",
      "on": "AlbumId",
      "how": "inner"
    },
    "save_as": "df_2"
  }  ,
  {
    "step": 3,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "df_2",
      "df_b": "invoice_line",
      "on": "TrackId",
      "how": "inner"
    },
    "save_as": "df_3"
  },
  {
    "step": 4,
    "action": "OP_GROUP_BY",
    "parameters": {
      "df_a": "df_3",
      "group_by_columns": ["artist.Name"],
      "agg_functions": ["SUM(invoice_line.Quantity * track.UnitPrice) AS total_revenue"]
    },
    "save_as": "df_4"
  },
  {
    "step": 5,
    "action": "OP_ORDER",
    "parameters": {
      "df_a": "df_4",
      "columns": ["total_revenue"],
    

In [ ]:
#008 analysis choice selected plan in execute plan (chat):  
[{'step': 1, 'action': 'define_global_query', 'parameters': {'query': 'Artistas que mas han facturado?', 'context_data': ''}, 'save_as': 'response_1'}, {'step': 2, 'action': 'ask_db_expert', 'parameters': {'query': 'Top artists by total revenue', 'context_data': ''}, 'save_as': 'response_2'}]

#99 ask db expert query  Top artists by total revenue
#debug begin init db  17:45:32
#debug begin analyze doc   17:45:32
###93 question  Top artists by total revenue
###94 error  Invalid json output: [
  {
    "step": 1,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "artist",
      "df_b": "album",
      "on": "ArtistId",
      "how": "inner"
    },
    "save_as": "df_1"
  },
  {
    "step": 2,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "df_1",
      "df_b": "track",
      "on": "AlbumId",
      "how": "inner"
    },
    "save_as": "df_2"
  },
  {
    "step": 3,
    "action": "OP_JOIN",
    "parameters": {
      "df_a": "df_2",
      "df_b": "invoice_line",
      "on": "TrackId",
      "how": "inner"
    },
    "save_as": "df_3"
  },
  {
    "step": 4,
    "action": "OP_GROUP_BY",
    "parameters": {
      "df_a": "df_3",
      "group_by_columns": ["artist.Name"],
      "agg_functions": ["SUM(invoice_line.Quantity * track.UnitPrice) AS total_revenue"]
    },
    "save_as": "df_4"
  },
  {
    "step": 5,
    "action": "OP_ORDER",
    "parameters": {
      "df_a": "df_4",
      "columns": ["total_revenue"],
      "ascending": [False]
    },
    "save_as": "df_5"
  },
  {
    "step": 6,
    "action": "OP_LIMIT",
    "parameters": {
      "df_a": "df_5",
      "limit": 10
    },
    "save_as": "df_6"
  }
]